In [42]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
)
import time

In [43]:
# adjust these constants, then run entire notebook.

path_to_model = "./2021-07-23.3/results/checkpoint"

In [37]:
test_parameters = {
        'max_length': 128,
        'temperature': 0.9,
        'no_repeat_ngram_size': 4,
        'do_sample': True,
        'top_k': 50}

two_prompts= ["""R: reel
M: 4/4
K: Edor
|: E2 B E d E B E |""",
             
      """R: jig
M: 6/8
K: Gmaj
|: G3 G A B |"""]


test_prompts = [
    
    # drowsy maggie - Reel, Edor
    """R: reel
M: 4/4
K: Edor
|: E2 B E d E B E |""",
    
    # kesh jig - Jig, Gmaj
    """R: jig
M: 6/8
K: Gmaj
|: G3 G A B |""",
    
    # john ryan's - Polka, Dmaj 
    """R: polka
M: 2/4
K: Dmaj
d d B/ c/ d/ B/ |""",
    
    # king of the fairies - Hornpipe
    """R: hornpipe
M: 4/4
K: Edor
|: B,2 | E D E F G F G A""",
    
    # inisheer - Waltz
    """R: waltz
M: 3/4
K: Gmaj
B2 B A B d |""",
    
    # the butterfly - Slip Jig
    """R: slip jig
M: 9/8
K: Emin
|: B2 E G2 E F3 |""",
    
    # banish misfortune - Dmix
    """R: jig
M: 6/8
K: Dmix
f e d c A G |""",
    
    # tam lin - Dmin, low register
    """R: reel
M: 4/4
K: Dmin
A,2 D A, F A, D A, |""",
    
    # cliffs of moher - Ador, high register
    """R: jig
M: 6/8
K: Adow
|: a3 b a g |""",
    
    # silver spear - no closing barline
    """R: reel
M: 4/4
K: Dmaj
A |: F A (3 A A A B A F A""",
    
    # unspecified tune types and meters
    "R: jig",
    "R: reel",
    "R: polka",
    "R: waltz",
    "R: hornpipe",
    "R: slip jig",
    "M: 4/4",
    "M: 6/8",
    "M: 3/4",
    "M: 2/4",
    "M: 9/8",
    
    # single notes
    "G",
    "e"
]


# for compiling html file
html_header = """<html>
<meta charset="UTF-8"/>
<script src="http://moinejf.free.fr/js/abcweb-1.js"></script>
<script src="http://moinejf.free.fr/js/snd-1.js"></script>
<style>svg {display:block}</style>
<title></title>
</head>
<body bgcolor="#faf0e6">"""

html_footer = """</body>
</html>"""

html_abc_prefix = """<script type="text/vnd.abc">
%abc-2.2
%%pagewidth 14cm
%%bgcolor white
%%topspace 0
%%composerspace 0
%%leftmargin 0.8cm
%%rightmargin 0.8cm

"""

html_abc_postfix = """</script>\n\n"""

In [38]:
tokenizer = AutoTokenizer.from_pretrained(path_to_model)
model = AutoModelForCausalLM.from_pretrained(path_to_model)

In [39]:
def generate(start_text = "a", number = 6, parameters = test_parameters):
    # encoding the input text
    input_ids = tokenizer.encode(start_text, return_tensors='pt')
    output = model.generate(input_ids, num_return_sequences = number, **parameters)
    return output

In [40]:
def test_suite(path = path_to_model,
               parameters = test_parameters,
               prompts = test_prompts,
               adjust_tabs = True):
    
    current_time = time.strftime("%Y-%m-%dt%H:%M:%S", time.localtime())
    txt_file_name = "test_results/{t}.txt".format(t = current_time)
    html_file_name = "test_results/{t}.html".format(t = current_time)
    print("running tests on model at {}".format(path))
    print("saving to {} and {}".format(txt_file_name, html_file_name))
    
    with open(txt_file_name, 'a') as txt_file, open(html_file_name, 'a') as html_file:
        
        html_file.write(html_header)
        
        txt_file.write('{}\n\n'.format(current_time))
        html_file.write('<p>{}</p>\n'.format(current_time))
        
        txt_file.write("path_to_model: {}\n\n".format(path_to_model))
        html_file.write("<p>path_to_model: {}</p>\n".format(path_to_model))
        
        txt_file.write("parameters:\n")
        html_file.write("<p>parameters:\n<ul>\n")
        for p in parameters:
            txt_file.write("    {}: {}\n".format(p, parameters[p]))
            html_file.write("<li>{}: {}\n</li>\n".format(p, parameters[p]))
        txt_file.write("\n=========\n\n")
        html_file.write("</ul>\n</p>\n")
        
        html_file.write("<ul>")
        for i, prompt in enumerate(prompts):
            html_file.write('<li><a href="#prompt{num}">Prompt {num} - {p}</a></li>\n'.format(num = i + 1, p = prompt))
        html_file.write("</ul>")
        
        prompt_count = 0
        test_count = 1
        for prompt in prompts:
            if adjust_tabs:
                prompt = prompt.replace('\n', '\t')
            prompt_count += 1
            print("running prompt {}: {}".format(prompt_count, prompt))
            
            prompt_counter_text = "prompt {}:\n".format(prompt_count)
            prompt_text = "prompt:\n{}\n\n\n".format(prompt)
            txt_file.write(prompt_counter_text)
            txt_file.write(prompt_text)
            html_file.write('<h3 id="prompt{}">{}</h3>'.format(prompt_count, prompt_counter_text))
            html_file.write("<p>{}<br>".format(prompt_text))
            
            output = generate(prompt)
            for setting in output:
                s = tokenizer.decode(setting)
                if adjust_tabs:
                    s = s.replace('\t', '\n')
                txt_file.write(s + "\n---\n")
                html_file.write("\nTest: {}\n".format(test_count))
                html_file.write(html_abc_prefix)
                html_file.write("X: {}\n".format(test_count))
                html_file.write(s.replace(" ", "") + "\n\n")
                html_file.write(html_abc_postfix)
                
                test_count += 1
            
            html_file.write("</p>")
            txt_file.write("\n---------\n\n")
            
        
        html_file.write(html_footer)

In [41]:
test_suite(adjust_tabs=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


running tests on model at ./2021-07-23.3/results/checkpoint
saving to test_results/2021-07-23t14:01:25.txt and test_results/2021-07-23t14:01:25.html
running prompt 1: R: reel	M: 4/4	K: Edor	|: E2 B E d E B E |


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


running prompt 2: R: jig	M: 6/8	K: Gmaj	|: G3 G A B |


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

running prompt 21: M: 9/8


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


running prompt 22: G


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


running prompt 23: e


In [ ]:
output = generate('G A B A d d |')
for x in output:
    print(tokenizer.decode(x), '\n')

In [64]:
test_suite(prompts = two_prompts)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


running tests on model at ./2021-07-20.1/results/checkpoint-21610
saving to test_results/2021-07-23t13:52:01.txt and test_results/2021-07-23t13:52:01.html
running prompt 1: R: reel	M: 4/4	K: Edor	|: E2 B E d E B E |


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


running prompt 2: R: jig	M: 6/8	K: Gmaj	|: G3 G A B |
